In [ ]:
from time import sleep
from tm1640 import TM1640

# Simple and elegant usage

In [ ]:
with TM1640(clk_pin=24, din_pin=23) as d:
    d.brightness = 0
    d.write_text('HELLO')
    for i in [1,1,1,1,1, -1,-1,-1,-1,-1]:
        sleep(1)
        d.brightness += i

# Global object for the purposes of this notebook

In [ ]:
disp = TM1640(clk_pin=24, din_pin=23)

In [ ]:
disp.brightness = 1

# Showing some garbage

In [ ]:
disp.write_bytes(b'0123456789abcdef')

In [ ]:
disp.write_bytes([0xff, 0xef, 0b10000000, 0x63])

In [ ]:
disp.write_bytes([0, 0x63, 0x5c, 0], 8)

In [ ]:
import random
for i in range(64):
    disp.write_bytes(bytes(random.randrange(256) for b in range(16)))
    sleep(1 / (i+1))

# Testing different characters

In [ ]:
disp.write_text('.01.02..03')

In [ ]:
disp.write_text('0123456789 yYzZ')

In [ ]:
disp.write_text('aAbBcCdDeEfFgGhH')

In [ ]:
disp.write_text('iIjJkKlLmMnNoOpP')

In [ ]:
disp.write_text('qQrRsStTuUvVwWxX')

In [ ]:
disp.write_text('~!@#$%^&*()[]{}')

In [ ]:
disp.write_text('-_¯\'`"+=,./\\:;')

In [ ]:
disp.write_text('🯰🯱🯲🯳🯴🯵🯶🯷🯸🯹⁐ニ≡‾|‖')

In [ ]:
disp.write_text('⌈⌉⌊⌋⎾⏋⎿⏌⌜⌝⌞⌟⌌⌍⌎⌏')

In [ ]:
disp.write_text('⊦⊢⊣⎡⎢⎣⎤⎥⎦')

In [ ]:
disp.write_text('⊏⊑⊐⊒⊓⊔⋂⋃Πμ')

# Some progress bar simulations

In [ ]:
import textwrap
def animate(textarea):
    for i in textwrap.dedent(textarea).strip().splitlines():
        disp.write_text(i)
        sleep(1)

In [ ]:
animate('''
    [   ]
    [⁐  ]
    [⁐⁐ ]
    [⁐⁐⁐]
''')

In [ ]:
animate('''
    [   ]
    [.   ]
    [..  ]
    [... ]
    [....]
''')

In [ ]:
animate('''
    ⌈   ⌉
    [   ⌉
    [.   ⌉
    [._  ⌉
    [._.  ⌉
    [._._ ⌉
    [._._. ⌉
    [._._._⌉
    [._._._.⌉
    [._._._.]
''')

In [ ]:
animate('''
    [  ]
    E  ]
    8  ]
    8| ]
    8E ]
    88 ]
    88|]
    88E]
    888]
    8880
    8888
''')

In [ ]:
animate('''
    ⎢   ⎥
    ‖   ⎥
    ‖⎢  ⎥
    ‖‖  ⎥
    ‖‖⎢ ⎥
    ‖‖‖ ⎥
    ‖‖‖⎢⎥
    ‖‖‖‖⎥
    ‖‖‖‖‖
''')

In [ ]:
from collections import namedtuple

PBS = namedtuple('PBS', 'left middle right half full')  # PBS = Progress Bar Style

progress_bar_styles = [
    # ⌊ _ _ _ ⌋
    PBS(0b_0011000, 0b_0001000, 0b_0001100, 0b0100000, 0b0100010),
    # ⌊._._._.⌋
    PBS(0b10011000, 0b10001000, 0b_0001100, 0b0100000, 0b0100010),
    # ⌈ ¯ ¯ ¯ ⌉
    PBS(0b_0100001, 0b_0000001, 0b_0000011, 0b0010000, 0b0010100),
    # ‖.‖.‖. . .
    PBS(0b10000000, 0b10000000, 0b10000000, 0b0110000, 0b0110110),
    # ‖.‖.‖._._.
    PBS(0b10001000, 0b10001000, 0b10001000, 0b0110000, 0b0110110),
]


def progress_bar(total, filled, theme):
    assert total >= 1
    assert isinstance(total, int)
    bits = total * 2
    marks = round(filled * 2)

    buffer = [0] * total
    buffer[0] |= theme.left
    buffer[-1] |= theme.right
    for i in range(total):
        if i > 0 and i + 1 < total:
            buffer[i] |= theme.middle
        if i * 2 + 1 < marks:
            buffer[i] |= theme.full
        elif i * 2 < marks:
            buffer[i] |= theme.half
    return bytes(buffer)

In [ ]:
def byteanimate(iterable, delay=1.0):
    disp.write_text('')
    for i in iterable:
        disp.write_bytes(i)
        sleep(delay)

In [ ]:
for total in [1, 2, 3, 4, 8]:
    for theme in progress_bar_styles:
        byteanimate(
            (progress_bar(total, i / 2, theme) for i in range(0, 2 * total + 1)),
            1 / total
        )

In [ ]:
# This looks like a stereo VU meter.
def double_progress_bar(total, top, bottom):
    assert total >= 1
    assert isinstance(total, int)
    bits = total * 2
    tops = round(top * 2)
    bots = round(bottom * 2)

    buffer = [0] * total
    for i in range(total):
        if i * 2 + 1 < tops:
            buffer[i] |= 0b0100010
        elif i * 2 < tops:
            buffer[i] |= 0b0100000
        if i * 2 + 1 < bots:
            buffer[i] |= 0b0010100
        elif i * 2 < bots:
            buffer[i] |= 0b0010000
    return bytes(buffer)

In [ ]:
import math
for total in [1, 2, 3, 8]:
    for theme in progress_bar_styles:
        byteanimate(
            (double_progress_bar(
                total,
                (math.sin(2 * i * math.tau / 64) + 1) / 2 * total,
                (math.cos(2 * i * math.tau / 64) + 1) / 2 * total
            ) for i in range(0, 64)),
            1 / 64
        )